In [ ]:
import os
import sys
import pandas as pd

# Notebookがあるディレクトリから1階層上を取得
base_dir = os.path.abspath("..")

# modulesパッケージをimportするために、modulesフォルダの親ディレクトリをPythonパスへ追加
sys.path.append(base_dir)

# ここで、modules/ の中に __init__.py があり、
# dictionary.py や classify.py が置いてあることを想定
import modules.dictionary as dict
from modules.classify import classify_keywords

# input_data フォルダに移動したい場合
input_data_dir = os.path.join(base_dir, "input_data")
os.chdir(input_data_dir)
print("current directory:", os.getcwd())

In [8]:
# target word(s) : class(es) assigned

override_map_2020 = {

    # ----内科・外科----
    ("訪問", "訪問診療", "訪問診療科", "内", "小内", "往診専門", "健康診", "男内", "女内", "内小", "内（薬）", "(新)内", "新生内", "内（兼", "漢方外来", "女性内"): ["一般内科"],
    ("成人病検査のみ", "健康指導", "内（採血）", "健診、人間", "健診", "健診等"): ["一般内科"],
    ("漢", "漢方", "漢方専門", "漢方科", "漢方（思）", "漢方（不）", "漢方医学科", "漢方診療科", "漢小", "漢方内", "漢方）"): ["一般内科"], 
    ("総合診療", "総合診療科", "総合", "内．外", "女性", "男性"): ['一般内科', '一般外科'],
    ("基本診療", "基本診療科", "全科"): ['一般内科', '一般外科'],
    ("一般外", '外'): ['一般外科'],

     # ----消化器内科・外科----
     ("内視内", "鏡内", "内（鏡）", "内視", "内鏡内"): ["消化器内科"],
    ("消", "消科", "消化", "消化器", "消火器科", "消化器科", "消化器科（胃）", "「消化器", "（消化器）", "消（胃）", "胃腸科 "): ["消化器内科", "消化器外科"],
    ("胃", "胃腸", "胃腸科", "胃腸下科", "胃腸器科", "消・胃腸科", "消胃", "胃腸科（消化器科）", "消化器科∧胃腸科∨"): ["消化器内科", "消化器外科"],
    ("呼胃"): ["呼吸器内科", '呼吸器外科', "消化器内科", "消化器外科"], 
    ("食道科", "食道", "食"): ["消化器内科", "消化器外科"],
    ("消化器・胃腸科", "消化器胃腸科", "消化器科（胃腸科）", "消化器科 "): ["消化器内科", '消化器外科'],
    ("内視外", "鏡外", "内鏡外", "女内視外", "こう乳外", "腹外", "腹部外", "外(鏡)"): ["消化器外科"],

     # ----呼吸器内科・外科----
    ("呼", "呼吸", "呼吸器", "呼吸器科", "呼吸科", "呼吸器科 "): ["呼吸器内科", '呼吸器外科'],
    ("（呼感"): ["呼吸器内科", '呼吸器外科', "感染症内科"],

    # ----気管食道内科・外科----
    ("気管食道科", "気食", "気管食道", "気管科", "気管支科", "気道食道科", "気管", "喘息外来"): ["気管食道内科", "気管食道外科"],

     # ----循環器内科・外科----
    ("循環器科", "循環器", "循環科", "循", "循環", "循小", "（循）小", "循環器）"): ["循環器内科", "循環器外科"],
    ("循呼", "呼循"): ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],
    ("胃循", "消循"): ["消化器内科", "消化器外科", "循環器内科", "循環器外科"],
    "禁煙外来": ["循環器内科"],

    # ----心臓血管内科・外科----
    ("心血内", "血内", "内（血）"): ["循環器内科"],
    ("心臓", "心血", "心臓血", "心臓血管", "血管",): ["循環器内科", "循環器外科"],
    ("胸外", "胸部外"): ["循環器外科", "呼吸器外科"],

    # ----耳鼻咽喉----
    ("耳","鼻", "耳咽", "頭頸", "耳鼻咽", "頭頸部", "耳鼻咽喉科", "耳鼻咽喉科科", "耳鼻咽喉頭頸部科", "耳咽咽喉科", "耳咽いんこう科", "耳鼻咽喉", "耳鼻", "耳鼻科", "耳鼻いんこう科", "耳い", "耳鼻い", "耳鼻いん", "耳鼻いんこ", "耳鼻いんこう", "耳鼻いんこう科（耳鼻科）", "耳鼻いんこう喉科", "漢方耳鼻咽喉科", "頭部", "耳鼻咽頭科", "頸部"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],

    ("気管食道耳鼻いんこう科", "気管食道耳鼻咽喉科", "気管食道耳鼻咽喉"): ["気管食道内科", "気管食道外科", "耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("気管耳（気管呼吸器　耳鼻咽喉科）", "気管食道・耳鼻いんこう科", "気耳", "耳気", "気管食道・耳鼻咽喉科", "耳鼻いんこう科系その他"): ["気管食道内科", "気管食道外科", "耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],

    # ----大腸・肛門----
    ("肛内", "大肛内"): ["大腸・肛門内科"],
    ("肛外", "大肛外", "こう外", "大肛", "外肛外", "鏡内肛外", "外(こう)", "外こう"): ["大腸・肛門外科"],
    ("肛","こう門科", "肛門科", "こう門", "大腸肛門科", "外肛内", "肛門", "大腸", "こう"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("こ外"): ["大腸・肛門外科"],

    # ----精神・心療----
    "精": ["精神科"],
    ("精心内"): ["精神科", "心療内科"],#
    ("子どものこころ診療科", "子どものこころの診療科"): ['心療内科'],
    ("児精",  "老神"): ['精神科'],
    "心身医療科": ['心療内科'],
    ("心療", "老心療", "老心内", "女心内"): ['心療内科'],

    # ----脳血管----
    ("脳・血管内科"): ['神経内科'],

    ("脳神経科", "脳神", "脳神経", "内（脳・神）", "老年神経科", "神経" ,"神経科", "神経科 "): ["神経内科", "神経外科"],
    ("神内", "神内他", "脳神（脳卒中）", "［神内］"): ["神経内科"],
    ("神外", "脊外", "脳形"): ["神経外科"],
   
    # ----血液内科", "腫瘍内科----
    ("血液", "血", "血腫内", "老腫内"): ["血液内科", "腫瘍内科"],
    ("腫外"): ["腫瘍外科"],
    ("腫瘍"): ["血液内科", "腫瘍内科", "腫瘍外科"],

    # ----リハビリ----
    ("ﾃｰｼｮﾝ科", "−ション科"): ['リハビリテーション科'],
    "リバビリテーション科": ["リハビリテーション科"],

    # ----リウマチ----
    "リ内": ['リウマチ科'],

    # ----産婦人科----
    ("女性不妊症", "周内"): ['産婦人科'], # 周内 → 周産期内科 (Perinatal Internal Medicine)??
    
    # ----肝臓・胆嚢・膵臓----
    ("肝臓科", "肝臓", "肝"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("胆のう", "胆の", "胆", "胆嚢"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("膵臓", "膵"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("肝臓・胆のう・膵臓科", "肝胆"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("肝臓・消化"): ["肝・胆・膵内科", "肝・胆・膵外科", "消化器内科", "消化器外科"],

    # ----糖尿病・代謝科----
    ("糖内", "糖内他", "糖内内", "糖"): ["糖尿病科"], 
    ("代内", "代内内", "体謝", "代"): ["代謝内科"],
    # ["糖尿病科", "代謝内科", "糖尿病・代謝外科"],# typo of 代謝(taisha)?

    # ----小児科----
    ("小", "こ", "こ内", "小内", "新内", "内小", "循小", "小(新)", "(新)内", "（循）小", "新小", "(新)小", "新生内"): ["小児科"],
    "小外": ["小児外科"],

    # ----ペインクリニック系----
    "ペ": ['麻酔科', "整形外科"],

    # ----癌----
    ("が内", "腫内", "腫血内"): ["腫瘍内科"], # おそらく「がん内科」   Oncology/Internal Medicine for Cancer Treatment?
    ("化内", "化内", "薬内", "臨"): ['臨床検査科'], 

    # ----歯科----
    ("矯科", "矯", "小矯", "口外"): ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」 Orthodontics?
    "矯葉": ["歯科・口腔顎顔面外科"], # what 葉 means?
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],
    "頭": ['歯科・口腔顎顔面外科'],

    # ----泌尿器科"---
    ("生殖医療", "（生殖医療）", "生殖治療", "泌尿器科（生殖医療）", "泌尿器", "ひ尿科", "泌尿科", "［泌尿器科］", "神経泌尿器科", "神泌"): ["泌尿器内科", "泌尿器外科"],
    ("必尿器科", "泌", "ひ尿", "泌尿器科", "泌尿器科（人口透析）", "透析泌尿器科", "泌尿器科（透析）", "泌尿器科系その他", "泌尿器科（人工透析）", "泌尿器科(人工透析）", "ひ尿(人工透析)", "ひ尿器科", "泌尿器科(人工透析)", "漢方泌尿器科", "老年泌尿器科", "泌尿", "男性泌尿器科", "女泌"): ["泌尿器内科", "泌尿器外科"],
    ("ひ", "女ひ", "小ひ", "（女）泌", "泌(人工透析）", "女性泌", "女性泌尿器科", "泌尿科（人工透析）", "男泌"): ["泌尿器内科", "泌尿器外科"], # 「ひ」は泌尿器内科？, "泌尿器外科" Urology?
    ('腫瘍泌尿器科'): ["泌尿器内科", "泌尿器外科", "血液内科", "腫瘍内科", "腫瘍外科"],
    ("泌外"): ["泌尿器外科"],

    # ----血液・腫瘍----
    ("腫", "がん", "がん漢方"): ["腫瘍内科", "腫瘍外科"],

    # ----腎臓・内分泌----
    ("内分内", "内内", "内分", "内秘内", "分泌内", "分内", "分", "内秘", "脂内", "（脂)内", "脂", "高血圧内分内", "内泌内"): ["腎臓内科", "内分泌内科"],
    ("小内内"): ["腎臓内科", "内分泌内科", "小児科"], # おそらく「小児内分泌内科」 Pediatric Endocrinology?
    ("腎", "腎臓", "腎臓病", "腎臓科", "分泌", "腎泌"): ["腎臓内科", "内分泌内科", "腎臓外科", "内分泌外科"],
    ("乳腺内", "乳内", "内乳", "乳甲外"): ["腎臓内科", "内分泌内科"],
    ("乳腺", "「乳腺", "乳", "乳腺科", "胸部・乳"): ["内分泌内科", "内分泌外科"],
    ("消化器乳"): ["腎臓内科", "内分泌内科", "腎臓外科", "内分泌外科",  "消化器内科", "消化器外科"],
    ("糖代内", "糖内分内", "内代内", "分代内", "脂代内", "内泌代内", "脂代", "糖泌内", "内泌代", "糖内内泌内"): ["腎臓内科", "内分泌内科", "糖尿病科", "代謝内科"],
    ("甲状腺診療科", "甲状腺", "甲状腺科"): ["腎臓内科", "内分泌内科", "腎臓外科", "内分泌外科"],
    "血液浄化": ["腎臓内科", "内分泌内科"],
    ("透", "透析", "透析科", "人工透",  "人工透析", "人工透析科", "血液透析",  "人透", "腎透", "血透",  "透内", "人口透析科", "通院人工透析診療所", "循環器科（透析）", "透科", "腎（人工透析）", "血液透析科", "腹膜透析）", "外（人透）", "泌(透）", "泌(人工透析)") : ["腎臓内科"],
    ("ひ（透）", "ひ(透)", "ひ(透）", '腎ひ', '腎（透）', '泌（人工透析）', "泌（透析）", "泌（人透）", "泌（透）", "腎臓泌尿器科"): ["腎臓内科", "内分泌内科", "泌尿器内科", "泌尿器外科"], # 「ひ」は泌尿器内科？, "泌尿器外科"ただひらがなで表現する意味がわからない。「透」の字があることから、透析？ Dialysis (Nephrology-related department)?
    "高内": ["腎臓内科", "内分泌内科"], # おそらく「高血圧・内分泌内科」 Hypertension and Endocrinology?
    "小腎": ['小児科', "腎臓内科", "内分泌内科"], # おそらく 小児腎臓内科 or 小児腎臓外科 Pediatric Nephrology or Pediatric Kidney Surgery?
    "内高精": ["腎臓内科", "内分泌内科", "泌尿器内科", "泌尿器外科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)    Endocrinology, Hypertension, and Reproductive Medicine / Reproductive Medicine (related to male infertility?)
    ("乳内分外", "乳内外", "乳・内外", "乳外", "内分外", "甲外", "内泌", "透外", "乳泌外", "内泌外", "外（乳）", "分外"): [ "内分泌外科"], # おそらく 乳腺内分泌内科 or 乳腺内分泌外科 Breast Endocrinology or Breast Endocrine Surgery?

    # ----アレルギー----
    ("ア内", "ｱ", "ア科", "ｱ科", "ア", "（ア）内"): ["アレルギー科"],
    # ----病理診断科----
    ("理", "臨検", "検診．検査業務"): ['病理診断科'], #???
    "成分献血": ['病理診断科'],#

    # ----others====
    ("疼", "疼痛", "疼内"): ['麻酔科', '整形外科', 'リハビリテーション科'],
    ("薬物療法"): ["臨床検査科", "精神科", "緩和ケア科"],
    ("脊椎", "脊髄外", "脊椎外", "脊椎髄外", "股関節外来", "側弯外来") : ['整形外科'],
    ("もの忘れ科"): ['精神科'],
    ("リ", "ﾘ", "り"): ['リハビリテーション科', 'リウマチ科'], # depends. check the context.
    ("思春", "思春期"): ["小児科", "産婦人科", "腎臓内科", "内分泌内科", "精神科"],
    ("形外", "形"): ['形成外科・美容外科'],
    ("腎泌尿器科", "腎臓泌尿器", "腎・泌尿器科（人工透析）"): ["泌尿器内科", "泌尿器外科", "腎臓内科", "腎臓外科"],

    "小外形泌こう": ['小児外科', '形成外科・美容外科', '泌尿器内科', "泌尿器外科", '大腸・肛門内科', '大腸・肛門外科'],
    "形泌": ['形成外科・美容外科', '泌尿器内科', "泌尿器外科"],
    "乳外肛外": ["腎臓外科", "内分泌外科", '大腸・肛門外科'],
    "静瘤外": ['心臓・血管外科', '循環器外科'],
    "胃．消．肛": ['消化器内科', '消化器外科', '大腸・肛門内科', "大腸・肛門外科"],

    "老年": ['老年科'],

    ("精尿病", "糖内(内分　代)", "(糖)内"): ["糖尿病科"],
    ("糖代分内"): ["糖尿病科", "代謝内科", "内分泌内科"],
    ("内分代内"): ["内分泌内科", "代謝内科"],
    ("内分糖", "糖内内泌", "糖内乳外", "糖分内"): ["内分泌内科", "糖尿病科"],
    ("糖代", "糖内代内", "糖脂代内", "糖内脂代内", "内糖代", "糖内泌内"): ["糖尿病科", "代謝内科"],
    ("血内糖", "糖血内"): ["血液内科", "糖尿病科"],
    ("糖内鏡内"): ["糖尿病科", "消化器内科"],
    ("肝臓・糖"): ["肝・胆・膵内科", "肝・胆・膵外科", "糖尿病科"],



    ("外．内", "内外", "外内"): ['一般外科', '一般内科'],
    "内．形．泌": ['一般内科', '形成外科・美容外科', '泌尿器内科', "泌尿器外科"],

    ("性内", "性", "感"): ['感染症内科'],

    ("殖医外", "女内殖医外"): ['泌尿器内科', "泌尿器外科", '産婦人科'],

    ("内．泌", "泌内"): ['一般内科', '泌尿器内科', "泌尿器外科"],
    "内．心内": ['一般内科', '心療内科'],

    ("脳"): ["神経外科", "神経内科"],
    ("脳内"): ["神経内科"],

    ("ア耳", "耳ア"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科", "アレルギー科"],
    ("耳気ア"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科", "アレルギー科", "気管食道内科", "気管食道外科"],

    ("脳心内"): ["神経内科", "循環器内科"],

    ("感ひ"): ["皮膚科", "感染症内科"],
    ("小外小ひ"): ["小児外科", "小児科", "皮膚科"],
    ("外ひ"): ["皮膚科"],
    ("外鏡"): ["消化器外科"],
    ("大こ外"): ["大腸・肛門外科"],

    ("外（化）"): ["臨床検査科", "一般外科"],

    ("内．糖内"): ["一般内科", "糖尿病科"],
    ("耳鼻いんこう科系その他"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("内視鏡外"): ["消化器外科"],

    ("心内(漢方）"): ["心療内科"],

    ("小児神経心療内科"): ["心療内科"],
    ("腎内（透）血管外"): ["腎臓内科", "循環器外科"],
    ("静瘤外"): ["循環器外科"],
    ("泌尿器科（男性不妊治療）", "男性不妊治療外科", "泌尿器科（人工透析　不妊治療）"): ["泌尿器内科", "泌尿器外科"],

    ("腎内（血液透析）", "透析内科（血液透析　腹膜透析）"): ["腎臓内科"],

    ("空港の同一ビル内での水平移動", "049-283-2133（女子栄養大学事務室）"): ["分類不可"],

    #250407update
    ("漢内", "女漢内", ""): ["漢方・中国医学"],
    ("糖内漢内"): ["糖尿病科", "漢方・中国医学"],
    ("内（糖）"): ["糖尿病科"],
    ("性病科", "性病", "感染症", "性感染症", "エイズ検査", "性感", "性感染症科", "感染"): ["感染症内科"],
    ("血管内"): ["循環器内科"],
    ("性感染症泌尿器科", "不妊治療泌尿器科", "男性泌尿器科（不妊治療）", "泌尿器科（不妊治療）"): ["泌尿器内科", "泌尿器外科"],
    ("ﾍﾟｲﾝｸﾘﾆｯｸ", "ペ外", "ペ内", "ペイ", "ぺ", "ペ内ペ外", "ﾍﾟｲﾝ外", "女ペ内", "ﾍﾟｲﾝ内", "ぺ内", "内ぺ", "ぺ外"): ["ペインクリニック"],
    ("婦人科（不妊治療　生殖医療）泌尿器科"): ["産婦人科", "泌尿器内科", "泌尿器外科"],
    ("心内"): ["心療内科"]
}

In [9]:
# load data
uniq_dpt = pd.read_csv('2020_v2.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
uniq_dpt

['内',
 'リハ',
 '精',
 '心内',
 '小',
 '外',
 '整',
 '産婦',
 '耳',
 '皮',
 '脳神経内科',
 '循',
 '眼',
 '泌',
 '放',
 '麻',
 '神',
 '呼吸器内科',
 '消化器内科',
 '消化器外科',
 '循環器内科',
 '脳',
 '心外',
 '神内',
 '乳腺外科',
 '形',
 'リウ',
 nan,
 '呼外',
 'ペインクリニック外科',
 '歯',
 '矯歯',
 '歯外',
 '病理診断科',
 '呼吸器外科',
 '心臓血管内科',
 'アレ',
 '病理',
 '婦',
 '緩和ケア内科',
 '循内',
 '消外',
 '肛外',
 '消内',
 '整外',
 '脳外',
 '耳咽',
 '呼内',
 '人透',
 '消',
 '胃',
 '児童',
 '思春期精神科',
 '形外',
 '救急科',
 '血液内科',
 '呼',
 '腎臓内科',
 '救急',
 '糖尿病内科',
 '腫瘍内科',
 '透内',
 '乳外',
 '矯',
 '小歯',
 '内視鏡内',
 '肝臓内科',
 '放射線診断科',
 '放射線治療科',
 '頭頸部外科',
 '腫瘍外科',
 '病理診断',
 '胃腸内科',
 '外科',
 '産',
 '血管外科',
 '神経精神',
 '小外',
 '循環器外科',
 '人工透析内科',
 '糖尿病内分泌科',
 '脳神経外科',
 '肛',
 '神経精神科',
 '頭頸部',
 '耳鼻咽喉科',
 'ペインクリニック',
 '病',
 '肛門外科',
 '血液',
 '糖尿病代謝内科',
 '内（消化器\u3000循環器\u3000呼吸器\u3000内分泌\u3000代謝\u3000糖尿病\u3000胃\u3000大腸内視鏡\u3000老人\u3000人工透析\u3000漢方）',
 '外（消化器\u3000末梢血管\u3000肛門\u3000内分泌\u3000老人）',
 '性',
 'ペインクリニック内科',
 '肛門内科',
 '内分泌内科',
 '感染症内科',
 '大腸外科',
 '美容皮膚科',
 '緩和ケア外科',
 '胃腸外科',
 '内視鏡内科',
 '内視鏡外科',
 '糖尿病内分泌内科',
 '糖尿病',


In [10]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2020,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the results in a DataFrame
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
display(result_df)

,内科系,一般内科,内科_特殊問診,呼吸器内科,耳鼻咽喉・頭頸部内科,気管食道内科,消化器内科,大腸・肛門内科,循環器内科,神経内科,...,放射線科,漢方・中国医学,予防医学,在宅医療,救急・集中治療科,臨床検査科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,脳神経内科,内,女性内科,呼吸器内科,耳,呼吸気内科,消化器内科,肛,循,脳神経内科,...,放,内（消化器 循環器 呼吸器 内分泌 代謝 糖尿病 胃 大腸内視鏡 老人 人工透析 漢方）,人間ドック,リハビリテーション科（訪問）,救急科,臨床検査科,病理診断科,緩和ケア内科,,神
1,呼吸器内科,内科,老年内科，呼吸器内科，女性内科,呼内,耳咽,気管食道耳鼻いんこう科,消内,内（消化器 循環器 呼吸器 内分泌 代謝 糖尿病 胃 大腸内視鏡 老人 人工透析 漢方）,循環器内科,脳,...,放射線診断科,漢方内科,健康診断,訪問歯科,救急,化学療法内,病理,緩和ケア外科,,NaN
2,消化器内科,総合内科,男性内科,呼,頭頸部,気管食道科,消,外（消化器 末梢血管 肛門 内分泌 老人）,心臓血管内科,神内,...,放射線治療科,漢内,予防歯科,訪問リハ,救,化学療法,病理診断,緩和ケ,,病
3,循環器内科,一般内科,男性内科（血液）,内（消化器 循環器 呼吸器 内分泌 代謝 糖尿病 胃 大腸内視鏡 老人 人工透析 漢方）,耳鼻咽喉科,気管食道内科,胃,肛門内科,循内,脳内,...,放射線診療科,漢方内科，漢方精神科，漢方･診療内科,外科（健康診断）,専ら在宅,ペインクリニック内科・救急科,臨床検,病診,緩和,,心
4,心臓血管内科,漢方外来,漢方女性内科,呼吸気内科,頭頸,喘息外来,内視鏡内,肛門,内（消化器 循環器 呼吸器 内分泌 代謝 糖尿病 胃 大腸内視鏡 老人 人工透析 漢方）,小児神経内科,...,放射線,アレルギー内科、消化器内科､漢方内科,健診、人間ドック,土井歯科医師：訪問診療専門,内．小．救,臨床検査課,病理診,緩和ケア科,,気
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,脂肪代謝内科,,,,,,,,,,...,,,,,,,,,,
482,循環器内科（透）,,,,,,,,,,...,,,,,,,,,,
483,内科（消化器 内視鏡 肝臓 糖尿病）,,,,,,,,,,...,,,,,,,,,,
484,生活習慣病内科,,,,,,,,,,...,,,,,,,,,,


In [11]:
result_df.to_csv('../classi_results/2020_classified.csv', encoding='utf-8', index=True)

In [12]:
result_df.to_excel('../classi_results/2020_classified.xlsx', index=True)